# Import Libraries

In [18]:
from langchain.llms.openai import OpenAIChat
from langchain.prompts.prompt import PromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain, SequentialChain
from langchain.globals import set_debug
from dotenv import load_dotenv

set_debug(True)  # Enable debug mode
load_dotenv()  # Load the variables from .env file

True

# LLMChain

Chain is an encapsulation of Model, prompt, and optionally a parser.

In [7]:
model = OpenAIChat(temperature=0.9)
prompt = PromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

chain = LLMChain(llm=model, prompt=prompt)

/home/mmostafa/miniconda3/envs/aienv10/lib/python3.10/site-packages/langchain/llms/openai.py:811: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


In [9]:
chain("Potato Chips")

[chain/start] [1:chain:LLMChain] Entering Chain run with input:
{
  "product": "Potato Chips"
}
[llm/start] [1:chain:LLMChain > 2:llm:OpenAIChat] Entering LLM run with input:
{
  "prompts": [
    "What is the best name to describe     a company that makes Potato Chips?"
  ]
}
[llm/end] [1:chain:LLMChain > 2:llm:OpenAIChat] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "The Crunch Factory",
        "generation_info": null
      }
    ]
  ],
  "llm_output": {
    "token_usage": {
      "prompt_tokens": 22,
      "completion_tokens": 3,
      "total_tokens": 25
    },
    "model_name": "gpt-3.5-turbo"
  },
  "run": null
}
[chain/end] [1:chain:LLMChain] s] Exiting Chain run with output:
{
  "text": "The Crunch Factory"
}


{'product': 'Potato Chips', 'text': 'The Crunch Factory'}

# SimpleSequentialChain

Chains can be connected to each other. The output of the previous chain is the input of the next chain. The `SimpleSequentialChain` is a simple chain that can be used to connect multiple chains.

In [17]:
# This is a simpler method to create a template.
prompt_1 = PromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

chain_1 = LLMChain(llm=model, prompt=prompt_1)

In [12]:
prompt_2 = PromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)

chain_2 = LLMChain(llm=model, prompt=prompt_2)

In [15]:
simple_chain = SimpleSequentialChain(chains=[chain_1, chain_2])

In [16]:
simple_chain("Potato Chips")

[chain/start] [1:chain:SimpleSequentialChain] Entering Chain run with input:
{
  "input": "Potato Chips"
}
[chain/start] [1:chain:SimpleSequentialChain > 2:chain:LLMChain] Entering Chain run with input:
{
  "product": "Potato Chips"
}
[llm/start] [1:chain:SimpleSequentialChain > 2:chain:LLMChain > 3:llm:OpenAIChat] Entering LLM run with input:
{
  "prompts": [
    "What is the best name to describe     a company that makes Potato Chips?"
  ]
}
[llm/end] [1:chain:SimpleSequentialChain > 2:chain:LLMChain > 3:llm:OpenAIChat] [1.90s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "\"Crisp Delights\"",
        "generation_info": null
      }
    ]
  ],
  "llm_output": {
    "token_usage": {
      "prompt_tokens": 22,
      "completion_tokens": 6,
      "total_tokens": 28
    },
    "model_name": "gpt-3.5-turbo"
  },
  "run": null
}
[chain/end] [1:chain:SimpleSequentialChain > 2:chain:LLMChain] [1.91s] Exiting Chain run with output:
{
  "text": "\"Crisp Delig

{'input': 'Potato Chips',
 'output': 'Crisp Delights offers a tantalizing selection of gourmet snacks and treats that are full of flavor and satisfaction.'}

# SequentialChain

it can be used to connect multiple chains non-sequentially, by specifying `output_key` that the next chain needs to receive.

In [36]:
# prompt template 1: translate to english
prompt_1 = PromptTemplate.from_template(
    "Translate the following review to english:" "\n\n{Review}"
)
# chain 1: input = Review and output = English_Review
chain_1 = LLMChain(llm=model, prompt=prompt_1, output_key="English_Review")

In [37]:
# prompt template 2: summarize
prompt_2 = PromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:" "\n\n{English_Review}"
)
# chain 2: input = English_Review and output = summary
chain_2 = LLMChain(llm=model, prompt=prompt_2, output_key="summary")

In [38]:
# prompt template 3: Know the language
prompt_3 = PromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input = Review and output = language
chain_3 = LLMChain(llm=model, prompt=prompt_3, output_key="language")

In [39]:
# prompt template 4: follow up message
prompt_4 = PromptTemplate.from_template(
    "Write a follow up response to the following \
    summary in the specified language: \
    \n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input = summary, language and output = followup_message
chain_4 = LLMChain(llm=model, prompt=prompt_4, output_key="followup_message")

In [40]:
# sequential_chain: input = Review
# and output = English_Review,summary, followup_message
sequential_chain = SequentialChain(
    chains=[chain_1, chain_2, chain_3, chain_4],
    input_variables=["Review"],
    output_variables=["English_Review", "summary", "followup_message"],
)

In [41]:
french_customer_review = """\
Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. \
J'achète les mêmes dans le commerce et le goût est bien meilleur."
"""

In [43]:
sequential_response = sequential_chain(french_customer_review)

[chain/start] [1:chain:SequentialChain] Entering Chain run with input:
{
  "Review": "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur.\"\n"
}
[chain/start] [1:chain:SequentialChain > 2:chain:LLMChain] Entering Chain run with input:
{
  "Review": "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur.\"\n"
}
[llm/start] [1:chain:SequentialChain > 2:chain:LLMChain > 3:llm:OpenAIChat] Entering LLM run with input:
{
  "prompts": [
    "Translate the following review to english:\n\nJe trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur.\""
  ]
}
[llm/end] [1:chain:SequentialChain > 2:chain:LLMChain > 3:llm:OpenAIChat] [2.31s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "I find the taste mediocre. The foam does not hold, 

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for gpt-3.5-turbo in organization org-dnanFMzkqEPXh6Dgutwmyuw0 on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for gpt-3.5-turbo in organization org-dnanFMzkqEPXh6Dgutwmyuw0 on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..


[llm/end] [1:chain:SequentialChain > 8:chain:LLMChain > 9:llm:OpenAIChat] [36.56s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Réponse de suivi:\n\nCher(e) critique,\n\nNous vous remercions d'avoir partagé vos commentaires sur votre expérience d'achat de notre produit. Nous sommes désolés d'apprendre que vous avez été déçu(e) par le goût médiocre et la faible stabilité de la mousse. Votre opinion est importante pour nous, et nous souhaitons nous améliorer en conséquence.\n\nNous tenons à souligner que nous prenons toutes les mesures nécessaires pour garantir la qualité de nos produits, y compris leur goût et leur texture. Cependant, il peut arriver que des variations se produisent en fonction de plusieurs facteurs, tels que la manipulation ou le stockage du produit.\n\nNous aimerions mieux comprendre votre expérience et identifier la cause de ce problème. Pourriez-vous nous fournir plus de détails, tels que le numéro de lot ou la date d'achat, afin 

In [45]:
sequential_response

{'Review': 'Je trouve le goût médiocre. La mousse ne tient pas, c\'est bizarre. J\'achète les mêmes dans le commerce et le goût est bien meilleur."\n',
 'English_Review': "I find the taste mediocre. The foam does not hold, it's strange. I buy the same ones in stores and the taste is much better.",
 'summary': 'The reviewer was disappointed with the mediocre taste and lack of foam stability, contrasting it with a better taste experience from the same product purchased in stores.',
 'followup_message': "Réponse de suivi:\n\nCher(e) critique,\n\nNous vous remercions d'avoir partagé vos commentaires sur votre expérience d'achat de notre produit. Nous sommes désolés d'apprendre que vous avez été déçu(e) par le goût médiocre et la faible stabilité de la mousse. Votre opinion est importante pour nous, et nous souhaitons nous améliorer en conséquence.\n\nNous tenons à souligner que nous prenons toutes les mesures nécessaires pour garantir la qualité de nos produits, y compris leur goût et leur